Predicting the Stock Market Status of a Company using Vedar Lexicon for the stock market.

In [1]:
#importing libraries
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import warnings
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt 
# nltk.download('vader_lexicon')
import csv

In [ ]:
#Creating the Lexicon for stock Market
sia = SentimentIntensityAnalyzer()
stock_lex = pd.read_csv('lexicon_data/stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

positive = []
with open('lexicon_data/lm_positive.csv', 'r') as f:
     reader = csv.reader(f)
     for row in reader:
         positive.append(row[0].strip())
    
negative = []
with open('lexicon_data/lm_negative.csv', 'r') as f:
     reader = csv.reader(f)
     for row in reader:
         entry = row[0].strip().split(" ")
         if len(entry) > 1:
             negative.extend(entry)
         else:
             negative.append(entry[0])

final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

The 50 Company Names and its correspoding Link are stored in a DataFrame by reading it from the Excel Sheet provided.
The news articles are scrapped from the links provided. The corresponding dates of the news articles are scrapped and stored in a list.
The words in the article are compared with the lexicon of the stock market and a score of +2 is given whenever a positive word is detected in the article and a score of -2 for the negative words and a range of scores from -1 to 1 were given to the neutral words.
The overall sentiment score for the particular day was evaluated and stored in a list.


Note: Change the directories of the csv files if needed.

In [ ]:
data=pd.read_csv('dataset_for_company\company_dataset.csv','r')
links=data.iloc[:,1]
companies=data.iloc[:,0]
headlines=[]
for i in range(0,len(links)):
    page = requests.get(links[i])
    soup = BeautifulSoup(page.text, 'html.parser')
    about=soup.findAll('div',class_="MT15 PT10 PB10")
    about=str(about)
    a=re.findall('<strong>.*</strong>',about)
    for j in range(0,len(a)):
        a[j]=a[j][8:len(a[j])-9]
    headlines.append(a)

dates=[]
for i in range(0,len(links)):
    page = requests.get(links[i])
    soup = BeautifulSoup(page.text, 'html.parser')
    about=soup.findAll('p',class_="PT3 a_10dgry")
    about=str(about)
    a_d=re.findall("[0-9][0-9] [A-Z][a-z][a-z] 2019",about)
    dates.append(a_d)


words=list(sia.lexicon)
for i in range(0,len(words)):
    words[i]=words[i].lower()
weights=list(sia.lexicon.values())
sent_score=[[]]
sc=[]
ps=PorterStemmer()
for k in range(0,len(headlines)):
    score=0
    for k1 in range(0,len(headlines[k])):
        review = headlines[k][k1]
        review = review.lower()
        review = review.split()
        review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
        for i in range(0,len(words)):
            for j in range(0,len(review)):
                if(review[j]==words[i]):
                    score=score+weights[i]
        sc.append(score)
    sent_score[0].append(sc)
    sc=[]

A scatter plot example of the news data and the sentiment scores of 2 companies are plotted.
A score of more than +6 in the market is considered as a positive outcome and the performance of the company is good enough in the stock market. 
An overall score of more than 50 is considered do be a positive output and the company's current position in the market is good and stable.

In [ ]:
plt.figure(1)
plt.scatter(dates[1],sent_score[0][1])
plt.title(companies[1]+"\n"+"Overall sentiment score as of today=  "+str(sum(sent_score[0][1])))
plt.show()

plt.figure(2)
plt.scatter(dates[2],sent_score[0][2])
plt.title(companies[2]+"\n"+"Overall sentiment score as of today=  "+str(sum(sent_score[0][2])))
plt.show()